In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from dataset import ReasoningHashDataset
checkpoint_path = "model_20250205_103231/model_checkpoint_batch_50"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, trust_remote_code=True)
dataset = ReasoningHashDataset(
    tokenizer=tokenizer,
    num_samples=10000,  # Small number for testing
    hash_length=4,  # Shorter hashes for testing
    chains=[2, 3, 4],  # Simpler chain lengths
    vary_hash=True,
    num_chains=3,
    device="cuda",
    rl=True
)



In [2]:
dataset.get_eval_item(0)

('Map:\no7in=>alwr\no7in=>nm3e\nxp6q=>mitl\nfzjz=>oxfv\nalwr=>xp6q\nnm3e=>v2a7\n3hgn=>fzjz\no7in=>3hgn\nStart: o7in\nTask: Multiple hash chains are provided. Find the shortest chain and provide the end 4 char hash. Think hard in tag! Circle your answer in <circle>HERE</circle> after </think>\n-----\nSTART\n<think>Lets reconstruct the list:\nnm3e=>v2a7\no7in=>nm3e\nfzjz=>oxfv\nalwr=>xp6q\n3hgn=>fzjz\nxp6q=>mitl\no7in=>3hgn\no7in=>alwr\n</think>\n<circle>v2a7</circle>',
 [('o7in', 'alwr'),
  ('o7in', 'nm3e'),
  ('xp6q', 'mitl'),
  ('fzjz', 'oxfv'),
  ('alwr', 'xp6q'),
  ('nm3e', 'v2a7'),
  ('3hgn', 'fzjz'),
  ('o7in', '3hgn')],
 'o7in',
 'v2a7',
 'Map:\no7in=>alwr\no7in=>nm3e\nxp6q=>mitl\nfzjz=>oxfv\nalwr=>xp6q\nnm3e=>v2a7\n3hgn=>fzjz\no7in=>3hgn\nStart: o7in\nTask: Multiple hash chains are provided. Find the shortest chain and provide the end 4 char hash. Think hard in tag! Circle your answer in <circle>HERE</circle> after </think>\n-----\nSTART\n<think>')

In [23]:
tokenizer.decode(dataset[0]["target"])

'h6lc<|endoftext|><|endoftext|><|endoftext|>'

In [24]:
import torch as T

# Load tokenizer and model
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=T.bfloat16
)

# Set model to eval mode for inference
model.eval()

# Quick test to verify loading
# test_input = "Map:\n9lqz=>9h1e\nmvnz=>z0e6\nmvnz=>9lqz\nz0e6=>xtka\nmvnz=>7ck5\nxtka=>ati4\nStart: mvnz\nTask: Multiple hash chains are provided. Find the shortest chain and provide the end 4 char hash. Think hard in tag! Circle your answer in <circle>HERE</circle> after </think>\n-----\nSTART\n"
# input_ids = tokenizer(t, return_tensors="pt").to(model.device)

with T.no_grad():
    output = model.generate(
        input_ids=dataset[0]["input"]["input_ids"].unsqueeze(0),
        attention_mask=dataset[0]["input"]["attention_mask"].unsqueeze(0),
        max_new_tokens=400,
    )

print("Test output:", tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Test output: Map:
qo09=>h6lc
7t8s=>hj60
rnzf=>gy3g
52ic=>rnzf
qo09=>7t8s
qo09=>52ic
Start: qo09
Task: Multiple hash chains are provided. Find the shortest chain and provide the end 4 char hash. Think hard in tag! Circle your answer in <circle>HERE</circle> after </think>
-----
START
<think>52ic=>rnzf is not provided, so we can't find the shortest chain.
qo09=>h6lc is provided, so we can't find the shortest chain.
7t8s=>hj60 is provided, so we can't find the shortest chain.
qo09=>52ic is provided, so we can't find the shortest chain.
52ic=>rnzf is provided, so we can't find the shortest chain.
qo09=>7t8s is provided, so we can't find the shortest chain.
</circle>h6lc</think>
<circle>h6lc</circle>


In [32]:
tokenizer.eos_token_id

151643

In [16]:
from typing import List
from tqdm import tqdm
import torch

def generate_hash(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long).to(model.device)
    pad_token_id = tokenizer.eos_token_id
    
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        num_return_sequences=1,
        temperature=0.6,
        top_p=0.95,
        pad_token_id=pad_token_id,
        do_sample=True,
    )
    
    generated_text = tokenizer.decode(output[0])
    # slide to just generated part
    generated_text = generated_text[generated_text.index(prompt)+len(prompt):]
    # get whats in between <circle> and </circle>
    # check if it has circle
    try:
        if "<circle>" not in generated_text and "</circle>" not in generated_text:
            return "NO CIRCLE"
        circle = generated_text.split("<circle>")[1].split("</circle>")[0]
    except:
        return "NO CIRCLE"
    return circle

def evaluate_reasoning_hash(model, tokenizer, num_tests: int = 100, hash_length: int = 5, 
                            chains: List[int] = [3, 4, 5, 6], vary_hash: bool = True, num_chains: int = 4):
    model.eval()
    
    # Create evaluation dataset
    eval_dataset = ReasoningHashDataset(tokenizer, num_samples=num_tests, hash_length=hash_length, 
                                        chains=chains, vary_hash=vary_hash, num_chains=num_chains)
    
    correct_predictions = 0
    total_predictions = 0
    chain_accuracies = {}

    for _ in tqdm(range(len(eval_dataset)), desc="Evaluating Reasoning Hash", leave=False):
        full_text, hash_list, start, actual_target, prompt = eval_dataset.get_eval_item(_)
        predicted_target = generate_hash(model, tokenizer, prompt)

        predicted_target = predicted_target[:3].lower()
        actual_target = actual_target[:3].lower()

        total_predictions += 1
        if any(predicted_target[i:i+3] == actual_target[i:i+3] for i in range(len(predicted_target)-2)):
            correct_predictions += 1
        
        # Determine the chain length for this sample
        chain_length = ReasoningHashDataset.find_shortest_path(hash_list, start) + 1
        
        # Initialize the chain_length entry if it doesn't exist
        if chain_length not in chain_accuracies:
            chain_accuracies[chain_length] = {'correct': 0, 'total': 0}
        
        chain_accuracies[chain_length]['total'] += 1
        if predicted_target == actual_target:
            chain_accuracies[chain_length]['correct'] += 1
        else:
            print(f"@@@Failed with {predicted_target} and {actual_target}")

    overall_accuracy = correct_predictions / total_predictions
    for length in chain_accuracies:
        if chain_accuracies[length]['total'] > 0:
            chain_accuracies[length]['accuracy'] = chain_accuracies[length]['correct'] / chain_accuracies[length]['total']
        else:
            chain_accuracies[length]['accuracy'] = 0

    return overall_accuracy, chain_accuracies

In [25]:
evaluate_reasoning_hash(model, tokenizer,  num_tests=100, hash_length=4, chains=[6,5,4,3,2], vary_hash=True, num_chains=3)

Evaluating Reasoning Hash:   1%|          | 1/100 [00:02<03:45,  2.28s/it]

@@@Failed with k0n and zve


Evaluating Reasoning Hash:   3%|▎         | 3/100 [00:06<03:46,  2.33s/it]

@@@Failed with dpr and grc


Evaluating Reasoning Hash:   4%|▍         | 4/100 [00:09<03:44,  2.34s/it]

...

Evaluating Reasoning Hash:  97%|█████████▋| 97/100 [05:25<00:06,  2.26s/it]

@@@Failed with ghq and dgn


Evaluating Reasoning Hash:  99%|█████████▉| 99/100 [05:29<00:02,  2.29s/it]

@@@Failed with zp5 and dxn


(0.37,
 {3: {'correct': 11, 'total': 22, 'accuracy': 0.5},
  4: {'correct': 4, 'total': 32, 'accuracy': 0.125},
  5: {'correct': 5, 'total': 19, 'accuracy': 0.2631578947368421},
  2: {'correct': 17, 'total': 27, 'accuracy': 0.6296296296296297}})

In [17]:
evaluate_reasoning_hash(model, tokenizer,  num_tests=100, hash_length=4, chains=[6,5,4,3,2], vary_hash=True, num_chains=3)

Evaluating Reasoning Hash:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating Reasoning Hash:   1%|          | 1/100 [00:02<03:49,  2.32s/it]

@@@Failed with lke and dvh


Evaluating Reasoning Hash:   2%|▏         | 2/100 [00:04<03:58,  2.43s/it]

@@@Failed with qbv and hid


...

Evaluating Reasoning Hash:  96%|█████████▌| 96/100 [03:57<00:09,  2.48s/it]

@@@Failed with 61x and k4l


Evaluating Reasoning Hash:  99%|█████████▉| 99/100 [04:04<00:02,  2.43s/it]

@@@Failed with 3vn and a4k


(0.42,
 {5: {'correct': 8, 'total': 34, 'accuracy': 0.23529411764705882},
  3: {'correct': 14, 'total': 25, 'accuracy': 0.56},
  4: {'correct': 12, 'total': 26, 'accuracy': 0.46153846153846156},
  2: {'correct': 8, 'total': 15, 'accuracy': 0.5333333333333333}})

In [19]:
evaluate_reasoning_hash(model, tokenizer,  num_tests=100, hash_length=4, chains=[2,3,4,5,6], vary_hash=True, num_chains=3)

Evaluating Reasoning Hash:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating Reasoning Hash:   1%|          | 1/100 [00:02<04:05,  2.48s/it]

@@@Failed with mjr and yiq


Evaluating Reasoning Hash:   2%|▏         | 2/100 [00:04<03:49,  2.34s/it]

@@@Failed with 9qw and ssb


...

@@@Failed with h7c and q46


Evaluating Reasoning Hash:  99%|█████████▉| 99/100 [04:03<00:02,  2.53s/it]

@@@Failed with f55 and nhc


@@@Failed with nyt and b5b


(0.39,
 {4: {'correct': 8, 'total': 24, 'accuracy': 0.3333333333333333},
  3: {'correct': 8, 'total': 25, 'accuracy': 0.32},
  5: {'correct': 9, 'total': 29, 'accuracy': 0.3103448275862069},
  2: {'correct': 14, 'total': 22, 'accuracy': 0.6363636363636364}})